In [1]:
import numpy as np # linear algebra
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import math
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader

from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, GroupKFold

timm_path = "../input/timm-pytorch-image-models/pytorch-image-models-master"
import sys
sys.path.append(timm_path)
import timm

import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
torch.autograd.set_detect_anomaly(True)

In [3]:
class GeM2(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM2,self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

In [4]:
df  = pd.read_csv("../input/train-folds/train_folds.csv",index_col=0)

In [5]:
train_aug = A.Compose(
    [  

        A.Resize(512,512,p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=120, p=0.5),
        A.CLAHE(p=0.5),
        A.Cutout(num_holes=3, max_h_size=45, max_w_size=45, p=0.4),
     A.Normalize(p=1.0),
        ToTensorV2(p=1.0)
    ]
)
val_aug = A.Compose(
    [  

        A.Resize(width=512, height=512, p=1.0),
     A.Normalize(p=1.0),
        ToTensorV2(p=1.0)
    ]
)

In [6]:
class Shop(Dataset):
    def __init__(self,df,augs=None):
        self.df = df
        self.augs = augs
        
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        img_src = self.df.path.iloc[idx]
        image = cv2.imread(img_src)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.uint8)
        
        p_id =  self.df.posting_id.iloc[idx]
        
        if (self.augs):
            transformed = self.augs(image=image)
            image = transformed['image']
        
        label = self.df.iloc[idx].label_group	
        label =torch.tensor(label, dtype=torch.long)
        
        return image,label,p_id

In [7]:
class ArcModule(nn.Module):
    def __init__(self, in_features, out_features, s = 10, m = 0.3):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.kaiming_normal_(self.weight)

        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = torch.tensor(math.cos(math.pi - m))
        self.mm = torch.tensor(math.sin(math.pi - m) * m)
        
      

    def forward(self, inputs, labels=None):
        cos_th = F.linear(inputs, F.normalize(self.weight))
        cos_th = cos_th.clamp(-1, 1) 
        sin_th = torch.sqrt(1.0 - torch.pow(cos_th, 2)+ 1e-8)
        cos_th_m = cos_th * self.cos_m - sin_th * self.sin_m
        # print(type(cos_th), type(self.th), type(cos_th_m), type(self.mm))
        cos_th_m = torch.where(cos_th > self.th, cos_th_m, cos_th - self.mm)

        cond_v = cos_th - self.th
        cond = cond_v <= 0
        cos_th_m[cond] = (cos_th - self.mm)[cond]

        if labels.dim() == 1:
            labels = labels.unsqueeze(-1)
        onehot = torch.zeros(cos_th.size()).to(device)
        labels = labels.type(torch.LongTensor).to(device)
        onehot.scatter_(1, labels, 1.0)
        outputs = onehot * cos_th_m + (1.0 - onehot) * cos_th
        outputs = outputs * self.s
        return outputs

In [8]:
class Model(nn.Module):
    def __init__(self,output_size =11014 ,pretrained=False):
        super().__init__()
        self.op = output_size
        self.backbone = timm.create_model('seresnet50', features_only=True, pretrained=pretrained)
        self.gem2 = GeM2()
        self.fc1 = nn.Linear(2048,512)
        self.do = nn.Dropout(p=0.25)
        self.bn1 = nn.BatchNorm1d(2048)
        self.bn2 = nn.BatchNorm1d(512)
        self.pl = nn.PReLU()
        self.margin = ArcModule(in_features=512, out_features = self.op)
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)

    def forward(self,x,labels=None):
        y = self.backbone(x)
        #y1 = self.gem1(y[2])
        y2 = self.gem2(y[4])
        
        #y1 = y1.view(x.shape[0],-1)
        y2 = y2.view(x.shape[0],-1)
        y2 = self.bn1(y2)
        
        #concat = torch.cat((y1,y2),dim = 1)
        fc1 = self.do(self.fc1(y2))
        feat1 = self.bn2(fc1)
        feat2 =  F.normalize(feat1)
        if labels is not None:
            return self.margin(feat2, labels)
        else:
            return feat2

In [9]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
def train_one_epoch(train_loader,model,optimizer,criterion,e,epochs,scheduler):
    losses = AverageMeter()
    model.train()
    global_step = 0
    loop = tqdm(enumerate(train_loader),total = len(train_loader))
    
    for step,(image,labels,_) in loop:
        image = image.to(device)
        labels= labels.to(device)
        logitss = model(image,labels)
        batch_size = labels.size(0)
        loss  = criterion(logitss,labels)
        
        losses.update(loss.item(), batch_size)
        
        optimizer.zero_grad()
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(m.parameters(), 1000 )
        optimizer.step()
        scheduler.step() 
        global_step += 1
        
        loop.set_description(f"Epoch {e+1}/{epochs}")
        loop.set_postfix(loss = loss.item() ,stage = 'train')
        
        
    return losses.avg

In [10]:
model = Model()
WEIGHTS_FILE = "../input/serex-final/image_modelT1_256_seresnext.pth"
model.load_state_dict(torch.load(WEIGHTS_FILE))
model.to(device);

In [11]:
def fit(fold):
  
    df_train = df[(df.fold == 1) | (df.fold == 2)  | (df.fold == 3) ].reset_index(drop=True)
    df_valid = df[df.fold == fold].reset_index(drop=True)
    
    train_data = Shop(df_train,augs= train_aug)
    val_data   = Shop(df_valid,augs=val_aug)
    
    train_loader = DataLoader(train_data,shuffle=True,
                        num_workers=4,
                        batch_size=16,
                        drop_last=True,
                            pin_memory=True)
    
    '''val_loader = DataLoader(val_data,shuffle=False,
                        num_workers=4,
                            pin_memory=True,
                        batch_size=32)'''
    criterion= nn.CrossEntropyLoss()
    epochs = 8
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4 , weight_decay = 1e-4)
    
    num_train_steps = math.ceil(len(train_loader))
    warmup_epochs = 1
    num_warmup_steps= num_train_steps * warmup_epochs
    num_training_steps=int(num_train_steps * epochs)
    scheduler = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps = num_warmup_steps,num_training_steps =num_training_steps)
    
    best_acc = 0
    loop = range(epochs)
    for e in loop:
        
        train_loss = train_one_epoch(train_loader,model,optimizer,criterion,e,epochs,scheduler)
        print(f'For epoch {e+1}/{epochs}')
        print(f'average train_loss {train_loss}')
        
        '''val_loss = val_one_epoch(val_loader,model,optimizer,criterion,scheduler)
        print(f'avarage val_loss { val_loss }')'''
        
        
    
    
fit(0)

  0%|          | 0/1284 [00:00<?, ?it/s]

For epoch 1/8
average train_loss 11.424033932968092


  0%|          | 0/1284 [00:00<?, ?it/s]

For epoch 2/8
average train_loss 10.425644933620346


  0%|          | 0/1284 [00:00<?, ?it/s]

...

For epoch 6/8
average train_loss 7.653612400512458


  0%|          | 0/1284 [00:00<?, ?it/s]

For epoch 7/8
average train_loss 7.441354178193945


  0%|          | 0/1284 [00:00<?, ?it/s]

For epoch 8/8
average train_loss 7.361205210566892


In [12]:
torch.save(model.state_dict(),'imageT123_seresnext.pth') 
test = df[df.fold == 0].reset_index(drop = True)
a = Shop(test,augs=val_aug)
t_loader =  DataLoader(a,shuffle=False,
                        num_workers=4,
                        batch_size=32,
                        pin_memory=True)
def generate_test_features(test_loader):
    model.eval()
    bar = tqdm(test_loader)
    
    FEAS = []

    with torch.no_grad():
        for batch_idx, (images,_,_) in enumerate(bar):

            images = images.to(device)

            features = model(images)

            FEAS += [features.detach().cpu()]

    FEAS = torch.cat(FEAS).cpu().numpy()
    
    return FEAS
FEAS = generate_test_features(t_loader)

  0%|          | 0/215 [00:00<?, ?it/s]

In [13]:
from sklearn import metrics
import gc
preds = []
CHUNK = 900
#te = ar[:,1:]
print('Finding similar titles...')
CTS = len(test)//CHUNK
if len(test)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(test))
    print('chunk',a,'to',b)
    cts = metrics.pairwise_distances(FEAS[a:b],FEAS, metric='cosine')
    cts = 1-cts
    for k in range(b-a):
        IDX = np.where(cts[k,]>0.85)[0]
        o = test.iloc[IDX].posting_id.values
        preds.append(o)

#del te,dx,modelx
_ = gc.collect()

Finding similar titles...
chunk 0 to 900
chunk 900 to 1800
chunk 1800 to 2700
chunk 2700 to 3600
chunk 3600 to 4500
chunk 4500 to 5400
chunk 5400 to 6300
chunk 6300 to 6851


In [14]:
test['preds'] = preds
tmp = test.groupby('image_phash').posting_id.agg('unique').to_dict()
test['preds3'] = test.image_phash.map(tmp)
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score
def combine_for_sub(row):
    x = np.concatenate([row.preds, row.preds3])
    return ' '.join( np.unique(x) )

def combine_for_cv(row):
    x = np.concatenate([row.preds, row.preds3])
    return np.unique(x)
tmp = test.groupby('label_group').posting_id.agg('unique').to_dict()
test['target'] = test.label_group.map(tmp)
test['oof'] = test.apply(combine_for_cv,axis=1)
test['f1'] = test.apply(getMetric('oof'),axis=1)
print('CV Score =', test.f1.mean() )

CV Score = 0.6945106124135104
